In [ ]:
## Reenforcement AI model for Play Mario

In [ ]:
#    Step1: Setup A Mario Environment
#    Step2: Preprocess the game for applied AI
#    Step3: Train the model to play Mario
#    Step4: Test It out live

### Install needed packages 

In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
#pip install nes-py

In [ ]:
pip list


In [1]:
# import the game
import gym_super_mario_bros
# import the joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [2]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [3]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode='human')
env = JoypadSpace(env,SIMPLE_MOVEMENT)

c:\users\abhij\appdata\local\programs\python\python38\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\users\abhij\appdata\local\programs\python\python38\lib\site-packages\gym\envs\registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [4]:
env.action_space

Discrete(7)

In [5]:
env.observation_space.shape

(240, 256, 3)

In [6]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [ ]:
# Create a flag - restart or not
done = True
#Loop throuth each frame in the game
for step in range(100000):
    if done:
        # start the game
        state = env.reset()
    # do random action
    state, reward, done, info,_ = env.step(env.action_space.sample())
    # Show the game on the screen 
    env.render()
# Close the game
env.close()

In [7]:
SIMPLE_MOVEMENT[env.action_space.sample()]

['NOOP']

In [ ]:
#Preprocessing Environment
#!pip install stable-baselines3

In [8]:
#Implement frame stacker wrapper and grayscalling wrapper
from gym.wrappers import FrameStack, GrayScaleObservation

#Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

#Import Matplotlib
from matplotlib import pyplot as plt

In [9]:
# Grayscale Environment
env = GrayScaleObservation(env,keep_dim=True)

# Wrap inside the dummy Environment 
env = DummyVecEnv([lambda: env])

## stack the frames
env=VecFrameStack(env,4,channels_order='last')

c:\users\abhij\appdata\local\programs\python\python38\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [ ]:
#!pip install shimmy

In [15]:
state=env.reset()

TypeError: reset() got an unexpected keyword argument 'seed'

In [ ]:
state.shape

In [10]:
# Use matplotlib to show the game frame
plt.imshow(state[0]) 

NameError: name 'state' is not defined

In [ ]:
state = env.reset()

In [ ]:
state.shape

In [ ]:
plt.imshow(state[0])

In [ ]:
state=env.reset()

In [ ]:
state.shape

In [ ]:
state, reward, done, info,_ = env.step([env.action_space.sample()])

In [ ]:
plt.imshow(state[0])

In [ ]:
plt.figure(figure=(10,8))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

### Train the model 

In [ ]:
import os
#import PPO algorithm
from stable_baselines3 import PPO

# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
 class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True
        

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
model.save('thisisatestmodel')

### Test it out

In [ ]:
# Load model
model = PPO.load('./train/best_model_1000000')

In [ ]:
state = env.reset()

In [ ]:
# Start the game 
state = env.reset()
# Loop through the game
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()